<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/EventProfiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install QSTK

  Using cached https://files.pythonhosted.org/packages/8c/15/3298c4ee5d187a462883a7f80d7621a05e8b880a8234729e733769a3476f/QSTK-0.2.8.tar.gz
    100% |████████████████████████████████| 235kB 10.7MB/s 
  Running setup.py bdist_wheel for QSTK ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/18/00/6b/01cf34934bf6a54b9d6df1c7a40230e18be1fd05d28e97d5ef
  Running setup.py bdist_wheel for python-dateutil ... - done
  Stored in directory: /root/.cache/pip/wheels/3e/57/ae/3a3f171dc8c4

In [1]:
!pip install pandas_datareader

<h3> Fetching all tickers from sp500</h2>

In [0]:
def get_sp500_tickers():
  import bs4 as bs    
  import pickle    
  import requests    
  import lxml    
  resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')        
  soup = bs.BeautifulSoup(resp.text,'lxml')        
  table = soup.find('table', {'class': 'wikitable sortable'})        

  tickers = []

  for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

  return tickers    

res = get_sp500_tickers()

<h2>Writing code to fetch data from yahoo (use IEX for recent data) </h2>

In [0]:
def get_data_from_yahoo(start, end, ticker, source='yahoo'):
  import pandas_datareader.data as dr
  import numpy as np
  from datetime import datetime
  return dr.DataReader(symbol, 'source', start, end)

from QSTK import

<h2> Importing required lbraries.(This code is borrowed from Computational Investing. course code only works for past data)</h2>

In [0]:


import pandas as pd; pd.TimeSeries = pd.Series


import numpy as np
import math
import copy
import QSTK.qstkutil.qsdateutil as du
import datetime as dt
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkstudy.EventProfiler as ep
